In [377]:
from dash import Dash, html, dcc, dash_table
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import requests
from jupyter_dash import JupyterDash
import datetime
from pyxirr import xirr

app = JupyterDash(__name__)

In [378]:
post_data = {"idEspecie": "48454"}
data_json = requests.post("https://www.iamc.com.ar/Financial/GetEspecieTablaDatosRelacionados", data=post_data).json()["EspecieFlujoFondosList"]
df_flujo = pd.DataFrame(data_json)

In [379]:
app.layout = html.Div([
    html.Div(id="table-div"),
    dcc.Dropdown(
        options=[
        {'label': 'AE38', 'value': 48461},
        {'label': 'TX26', 'value': 48454},
        ],
        value=48461,
        id="bonos-dropdown"
    ),
    html.Div(id="tir-div"),
    html.Div(dcc.Input(id='input-on-submit', type='number', value=30)),
    html.Button('Submit', id='submit-val', n_clicks=0),
])

data_json = None
df_flujo = None
df_tir  = None

@app.callback(
    Output("table-div", "children"),
    Input("bonos-dropdown", "value"),
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate

    post_data = {"idEspecie": search_value}
    data_json = requests.post("https://www.iamc.com.ar/Financial/GetEspecieTablaDatosRelacionados", data=post_data).json()["EspecieFlujoFondosList"]
    df_flujo = pd.DataFrame(data_json)
    df_flujo.drop(df_flujo.tail(1).index,inplace=True)

    df_tir = pd.DataFrame(columns=["Fecha", "Flujo"])
    df_tir["Fecha"] = pd.to_datetime(df_flujo["FechaPago"], format="%d/%m/%Y")
    df_tir["Flujo"] = df_flujo["CashFlow"]
    
    return dash_table.DataTable(df_flujo.to_dict('records'), [{"name": i, "id": i} for i in df_flujo.columns], id="tabla", page_size=10)

@app.callback(
    Output("tir-div", "children"),
    Input('submit-val', 'n_clicks'),
    State('input-on-submit', 'value')
)
def update_output(n_clicks, precio):
    df_calculo = df_tir
    df_calculo.loc[-1] = [datetime.datetime.today(), -precio]
    # df_calculo.index = df_calculo.index + 1
    # df_calculo = df_tir.sort_index()

    # tir = "La tir es :" + str(xirr(df_calculo)*100) + " %"
    return precio #"La tir es :" + str(xirr(df_tir)*100) + " %"




In [380]:
app.run_server(mode='inline')